In [2]:
# Optimizar dataframe para la función 1

import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")
import pyarrow
import re
import ast


# Empiezo a trabajar con user_items para reducir su tamaño
df_b = pd.read_parquet('Tablas/user_items_limpio.parquet')

In [3]:
# Elimino las columnas que no voy a utilizar
df_b.drop(columns=['user_id', 'item_name'])

,item_id,playtime_forever
0,10,6
1,30,7
2,300,4733
3,240,1853
4,3830,333
...,...,...
3246347,304930,677
3246348,227940,43
3246349,388490,3
3246350,521570,4


In [4]:
# Sumo la cantidad de horas jugadas por item_id
a = df_b.groupby('item_id')['playtime_forever'].sum()
# Transformo la serie en dataframe
df_playtime = a.to_frame()
df_playtime = df_playtime.reset_index(drop=False)
df_playtime

,item_id,playtime_forever
0,10,17107858
1,20,960524
2,30,756375
3,40,154424
4,50,726545
...,...,...
10045,527570,2
10046,527810,2
10047,527890,1
10048,527900,44


In [5]:
# Leo el siguiente dataframe: user_review_final.csv
# df_ur1 = pd.read_csv('user_review_final.csv')
df_ur1 = pd.read_parquet('Tablas/user_review_final.parquet')
df_ur1.shape

(48226, 6)

In [6]:
# Elimino las columnas que no voy a utilizar
df_ur2 =df_ur1.drop(columns=['user_id', 'helpful', 'recommend', 'review' ])
# Elimino las filas donde posted_year = 0
df_ur3 = df_ur2.drop(df_ur2[df_ur2['posted_year'] == 0].index)
df_ur3.reset_index(drop=True, inplace=True)
df_ur3['item_id'] = df_ur3['item_id'].astype(int)
df_ur3


,item_id,posted_year
0,1250,2011
1,251610,2014
2,250320,2013
3,211420,2014
4,249130,2014
...,...,...
48221,332310,2015
48222,440,2014
48223,304930,2014
48224,265630,2015


In [7]:
df_ur3.dtypes

item_id        int32
posted_year    int32
dtype: object

In [8]:
# Realizo un join de df_ur3 y df_playtime utilizando la columna item_id
df_join1 = df_ur3.merge(df_playtime, on="item_id", how="inner")
df_join1

,item_id,posted_year,playtime_forever
0,1250,2011,29737670
1,1250,2015,29737670
2,1250,2014,29737670
3,1250,2013,29737670
4,1250,2014,29737670
...,...,...,...
42532,288370,2014,15166
42533,73010,2012,173848
42534,16600,2012,95057
42535,232950,2014,43267


In [9]:
# Cambio el tipo de dato de 'item_id' a entero
df_join1['item_id'] = df_join1['item_id'].astype(int)

In [10]:
# Leo el dataframe steam_games_output
df_sgo = pd.read_parquet('Tablas/steam_games_output_limpio.parquet')
df_sgo


,genres,title,id,developer,release_year
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018
...,...,...,...,...,...
68605,Indie,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68606,Racing,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68607,Simulation,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68608,Casual,EXIT 2 - Directions,658870,"xropi,stev3ns",2017


In [11]:
# Elimino del dataframe todas las filas donde en  la columna release_year aparece 'Dato no disponible'
df_sgo1 = df_sgo[df_sgo["release_year"] != "Dato no disponible"]

In [12]:
df_sgo1.reset_index(drop=True, inplace=True)
df_sgo1

,genres,title,id,developer,release_year
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018
...,...,...,...,...,...
68605,Indie,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68606,Racing,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68607,Simulation,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68608,Casual,EXIT 2 - Directions,658870,"xropi,stev3ns",2017


In [13]:
# Me quedo con las columnas 'genres' e 'id'
df_sgo1 = df_sgo1.drop(columns=['title', 'developer', 'release_year'])
df_sgo1

,genres,id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
68605,Indie,610660
68606,Racing,610660
68607,Simulation,610660
68608,Casual,658870


In [14]:
df_sgo1.rename(columns={'id': 'item_id'}, inplace=True)
df_sgo1

,genres,item_id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
68605,Indie,610660
68606,Racing,610660
68607,Simulation,610660
68608,Casual,658870


In [15]:
# Cambio el tipo de dato de item_id a entero
df_sgo1['item_id'] = df_sgo1['item_id'].astype(int)

In [16]:
df_join2 = df_sgo1.merge(df_join1, on="item_id", how="inner")
df_join2

,genres,item_id,posted_year,playtime_forever
0,Action,70,2015,2650946
1,Action,70,2014,2650946
2,Action,70,2015,2650946
3,Action,70,2014,2650946
4,Action,70,2011,2650946
...,...,...,...,...
90013,Action,80,2014,2469131
90014,Action,80,2013,2469131
90015,Action,80,2011,2469131
90016,Action,80,2014,2469131


In [17]:
# Agrupo el dataframe por 'genres' y obtengo el índice del valor máximo de 'playtime_forever', es decir del máximo de horas jugadas por género.

idx = df_join2.groupby('genres')['playtime_forever'].idxmax()


In [18]:
# Me quedo con una lista de los géneros 
result = df_join2.loc[idx, ['genres', 'posted_year', 'playtime_forever']]
result

,genres,posted_year,playtime_forever
10977,Action,2015,775784836
83673,Adventure,2015,152997644
54617,Audio Production,2014,390165
38995,Casual,2014,50333977
64867,Early Access,2015,80110135
73597,Education,2015,175684
72035,Free to Play,2015,122726853
138,Indie,2013,441871026
63515,Massively Multiplayer,2015,80110135
84999,RPG,2015,152997644


In [19]:
# El siguiente código filtra en el dataframe 'result' el año correspondiente al género que yo ingreso. Pruebo su funcionamiento ingresando un género cualquiera.
genres_filtro = "Simulation"
df_filtrado = result.loc[result["genres"] == genres_filtro, "posted_year"]

In [ ]:
# El código incluído en api_functions.py difiere levemente a partir de este punto. Incluye lo siguiente:
# if df_filtrado.empty:
#       return "Género inexistente. Los géneros válidos para ingresar son: Action, Adventure, Audio Production, Casual, Early Access, Education, Free to Play, Indie, Massively Multiplayer, RPG, Racing, Simulation, Software Training, Sports, Strategy, Utilities, Video Production, Web Publishing. RESPETAR MAYÚSCULAS Y ESPACIOS ENTRE LAS PALABRAS."
# else:
#        a = df_filtrado.to_string(index=False)
#        respuesta = f"{{Año de lanzamiento con más horas jugadas para Género {genero} : {a}}}"
#        return respuesta

In [20]:
print(df_filtrado.to_string(index=False))

2013
